## CONFIG

In [2]:
import pdfplumber
import re
import json
import os
import html
import PyPDF2
import pandas as pd
from bs4 import BeautifulSoup



from collections import Counter

from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [3]:
codice_galattico = "../Hackapizza Dataset/Codice Galattico/Codice Galattico.pdf"
manuale_cucina = "../Hackapizza Dataset/Misc/Manuale di Cucina.pdf"
menu_esempio = "../Hackapizza Dataset/Menu/Anima Cosmica.pdf"

domande = "../Hackapizza Dataset/domande.csv"
piatti = "../Hackapizza Dataset/Misc/dish_mapping.json"
pianeti = "../Hackapizza Dataset/Misc/Distanze.csv"

blog_1 = "../Hackapizza Dataset/Blogpost/blog_etere_del_gusto.html"
blog_2 = "../Hackapizza Dataset/Blogpost/blog_sapore_del_dune.html"


load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [4]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [5]:
def estrai_testo_pdf(percorso_pdf):
    with open(percorso_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        testo = ""
        for pagina in reader.pages:
            testo += pagina.extract_text()
    return testo

def estrai_testo_html(percorso_html):
    """Estrae il testo da un file HTML."""
    with open(percorso_html, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        return soup.get_text(separator=" ", strip=True)

## METODI

In [6]:
import json
from json_repair import repair_json

def estrai_relazioni(testo, relazioni_info, llm, driver):
    """
    Estrae entità e relazioni tra di esse da un testo.
    
    Args:
        testo: Il testo da cui estrarre le relazioni
        relazioni_info: Dizionario nella forma {'NOME_RELAZIONE': ['Tipo_Entità1', 'Tipo_Entità2']}
        llm: Modello di linguaggio da utilizzare
        driver: Driver Neo4j
        
    Returns:
        Dizionario contenente entità e relazioni estratte
    """
    risultati = {}
    entita_tipi = set()
    
    # Raccogli tutti i tipi di entità unici dalle relazioni
    for _, [tipo_entita1, tipo_entita2] in relazioni_info.items():
        entita_tipi.add(tipo_entita1)
        entita_tipi.add(tipo_entita2)
    
    # Estrai tutte le entità una sola volta per tipo
    for tipo_entita in entita_tipi:
        print(f"Estraendo entità di tipo '{tipo_entita}'...")
        entita = estrai_valorizzazioni(testo, tipo_entita, llm, driver)
        risultati[tipo_entita] = entita
    
    # Ora estrai le relazioni
    for nome_relazione, [tipo_entita1, tipo_entita2] in relazioni_info.items():
        print(f"Estraendo relazioni '{nome_relazione}' tra '{tipo_entita1}' e '{tipo_entita2}'...")
        
        # Estrai le relazioni tra entità1 e entità2
        relazioni = estrai_valorizzazioni_relazioni(
            testo, 
            nome_relazione, 
            tipo_entita1, 
            tipo_entita2, 
            risultati[tipo_entita1], 
            risultati[tipo_entita2], 
            llm, 
            driver
        )
        
        # Salva le relazioni nel risultato
        risultati[nome_relazione] = relazioni
        
    return risultati

def estrai_valorizzazioni(testo, categoria, llm, driver):
    """
    Estrae le valorizzazioni di entità di una specifica categoria dal testo.
    """
    # Genera un prompt basato sulle entità filtrate     
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e sui loro Menu, Piatti, Ingredienti, Tecniche e Licenze.
    
    Il testo che segue potrebbe contenere informazioni pertinenti a questa categoria: {categoria}
    In output voglio tutte le possibile istanze di questa categoria, come entità da caricare su neo4j.
    Per ogni istanza recupera:
        - NOME dell'istanza
        - eventuali PARAMETRI associati all'istanza
    Attento che i nomi delle chiavi devono essere identiche a quelle fornite.
    
    Restituisci SOLO il JSON valido e completo, senza formattazione markdown o decoratori.
    
    Testo:
    {testo}
    """
    
    # Chiamata al modello LLM per ottenere la valorizzazione delle entità
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('```json'):
        content = content[7:].strip()
    elif content.startswith('json'):
        content = content[4:].strip()
    
    # Tentativo di riparare il JSON
    try:
        valorizzazioni = json.loads(content)
    except json.JSONDecodeError as e:
        print(f"Errore nel parsing JSON per entità {categoria}: {str(e)}")
        try:
            valorizzazioni = json.loads(repair_json(content))
        except Exception:
            # Fallback con richiesta all'LLM
            prompt_repair = f"Correggi questo JSON per Neo4j e assicurati che sia valido e completo:\n{content}"
            repaired = llm.invoke(prompt_repair).content.strip()
            
            # Pulizia del JSON riparato
            if repaired.startswith('```') and repaired.endswith('```'):
                repaired = '\n'.join(repaired.split('\n')[1:-1])
            if repaired.startswith('```json'):
                repaired = repaired[7:].strip()
            elif repaired.startswith('json'):
                repaired = repaired[4:].strip()
                
            try:
                valorizzazioni = json.loads(repaired)
            except:
                print(f"Impossibile riparare il JSON per le entità di tipo {categoria}")
                valorizzazioni = []
    
    # Assicurati che il risultato sia una lista
    if not isinstance(valorizzazioni, list):
        valorizzazioni = [valorizzazioni]
    
    return valorizzazioni

def estrai_valorizzazioni_relazioni(testo, nome_relazione, tipo_entita1, tipo_entita2, entita1, entita2, llm, driver):
    """
    Estrae le relazioni specifiche tra entità di due tipi.
    """
    # Genera un prompt per estrarre le relazioni
    prompt = f"""
    Contesto: questo testo rientra in una collezione di documenti che parlano di Ristoranti nello spazio (su Pianeti) e sui loro Menu, Piatti, Ingredienti, Tecniche e Licenze.
    
    Il testo che segue potrebbe contenere informazioni su relazioni di tipo '{nome_relazione}' tra entità di tipo '{tipo_entita1}' e '{tipo_entita2}'.
    
    In output voglio tutte le relazioni di tipo '{nome_relazione}' come triple da caricare su neo4j.
    Per ogni relazione recupera:
        - ORIGINE: nome dell'entità di tipo '{tipo_entita1}'
        - DESTINAZIONE: nome dell'entità di tipo '{tipo_entita2}'
        - eventuali PARAMETRI associati alla relazione
    
    Ecco le entità di tipo '{tipo_entita1}' già identificate:
    {[e.get("NOME", "") for e in (entita1 if isinstance(entita1, list) else [entita1])]}
    
    Ecco le entità di tipo '{tipo_entita2}' già identificate:
    {[e.get("NOME", "") for e in (entita2 if isinstance(entita2, list) else [entita2])]}
    
    Restituisci SOLO il JSON valido e completo, senza formattazione markdown o decoratori.
    Il formato deve essere:
    [
      {{"ORIGINE": "nome_entita1", "DESTINAZIONE": "nome_entita2", "PARAMETRI": {{...parametri...}} }}
    ]
    
    Testo:
    {testo}
    """
    
    # Chiamata al modello LLM per ottenere la valorizzazione delle relazioni
    response = llm.invoke(prompt)
    content = response.content.strip()
    
    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('```json'):
        content = content[7:].strip()
    elif content.startswith('json'):
        content = content[4:].strip()
    
    # Tentativo di riparare il JSON
    try:
        relazioni = json.loads(content)
    except json.JSONDecodeError as e:
        print(f"Errore nel parsing JSON delle relazioni {nome_relazione}: {str(e)}")
        try:
            relazioni = json.loads(repair_json(content))
        except Exception:
            # Fallback con richiesta all'LLM
            prompt_repair = f"Correggi questo JSON per Neo4j e assicurati che sia valido e completo:\n{content}"
            repaired = llm.invoke(prompt_repair).content.strip()
            
            # Pulizia del JSON riparato
            if repaired.startswith('```') and repaired.endswith('```'):
                repaired = '\n'.join(repaired.split('\n')[1:-1])
            if repaired.startswith('```json'):
                repaired = repaired[7:].strip()
            elif repaired.startswith('json'):
                repaired = repaired[4:].strip()
                
            try:
                relazioni = json.loads(repaired)
            except:
                print(f"Impossibile riparare il JSON delle relazioni {nome_relazione}")
                relazioni = []
    
    # Assicurati che il risultato sia una lista
    if not isinstance(relazioni, list):
        relazioni = [relazioni]
    
    return relazioni

def carica_entita_e_relazioni_su_neo4j(risultati, relazioni_info, driver):
    """
    Carica entità e relazioni su Neo4j
    
    Args:
        risultati: Dizionario contenente entità e relazioni
        relazioni_info: Dizionario delle relazioni come riferimento
        driver: Driver Neo4j
    """
    # Crea un set di tutti i tipi di relazione
    tipi_relazione = set(relazioni_info.keys())
    
    with driver.session() as session:
        # Per ogni tipo di entità nel risultato
        for tipo_entita, entita_list in risultati.items():
            # Salta le chiavi che sono nomi di relazione
            if tipo_entita in tipi_relazione:
                continue
                
            # Carica le entità
            print(f"Caricando {len(entita_list) if isinstance(entita_list, list) else 1} entità di tipo {tipo_entita}...")
            
            entita_array = entita_list if isinstance(entita_list, list) else [entita_list]
            for entita in entita_array:
                nome = entita.get("NOME", "")
                if not nome:  # Salta entità senza nome
                    continue
                    
                parametri = entita.get("PARAMETRI", {})
                
                # Sanificazione dei parametri per Neo4j (escape di apici)
                sanitized_params = {}
                for k, v in parametri.items():
                    if isinstance(v, str):
                        sanitized_params[k] = v.replace("'", "\\'")
                    else:
                        sanitized_params[k] = v
                
                # Converti i parametri in formato per Cypher
                params_string = ", ".join([f"{k}: '{v}'" for k, v in sanitized_params.items()])
                
                # Query Cypher per creare l'entità
                query = f"""
                MERGE (e:{tipo_entita} {{nome: $nome}})
                """
                
                if params_string:
                    query += f"""
                    SET e += {{{params_string}}}
                    """
                
                try:
                    session.run(query, nome=nome)
                except Exception as e:
                    print(f"Errore nel caricamento dell'entità {nome}: {str(e)}")
        
        # Per ogni tipo di relazione nel risultato
        for tipo_relazione, relazioni in risultati.items():
            # Salta le chiavi che non sono tipi di relazione
            if tipo_relazione not in tipi_relazione:
                continue
                
            # Determina i tipi di entità coinvolti nella relazione
            tipo_origine, tipo_destinazione = relazioni_info[tipo_relazione]
            
            # Carica le relazioni
            print(f"Caricando {len(relazioni)} relazioni di tipo {tipo_relazione}...")
            
            for relazione in relazioni:
                origine = relazione.get("ORIGINE", "")
                destinazione = relazione.get("DESTINAZIONE", "")
                
                if not origine or not destinazione:  # Salta relazioni incomplete
                    continue
                    
                parametri = relazione.get("PARAMETRI", {})
                
                # Sanificazione dei parametri per Neo4j
                sanitized_params = {}
                for k, v in parametri.items():
                    if isinstance(v, str):
                        sanitized_params[k] = v.replace("'", "\\'")
                    else:
                        sanitized_params[k] = v
                
                # Converti i parametri in formato per Cypher
                params_string = ", ".join([f"{k}: '{v}'" for k, v in sanitized_params.items()])
                
                # Query Cypher per creare la relazione
                query = f"""
                MATCH (o:{tipo_origine} {{nome: $origine}}), (d:{tipo_destinazione} {{nome: $destinazione}})
                MERGE (o)-[r:{tipo_relazione}"""
                
                if params_string:
                    query += f" {{{params_string}}}"
                
                query += "]->(d)"
                
                try:
                    session.run(query, origine=origine, destinazione=destinazione)
                except Exception as e:
                    print(f"Errore nel caricamento della relazione {origine}-[{tipo_relazione}]->{destinazione}: {str(e)}")

# Funzione principale per processare un testo completo
def processa_testo_completo(testo, relazioni_info, llm, driver):
    """
    Processa un testo estraendo entità e relazioni e caricandole su Neo4j.
    
    Args:
        testo: Testo da processare
        relazioni_info: Dizionario della forma {'NOME_RELAZIONE': ['Tipo_Entità1', 'Tipo_Entità2']}
        llm: Modello di linguaggio
        driver: Driver Neo4j
        
    Returns:
        Dizionario contenente le entità e relazioni estratte
    """
    # Estrai entità e relazioni
    risultati = estrai_relazioni(testo, relazioni_info, llm, driver)
    
    # Carica tutto su Neo4j
    carica_entita_e_relazioni_su_neo4j(risultati, relazioni_info, driver)
    
    return risultati

## MAIN

In [8]:
with open('entita_filtrate.txt', 'r', encoding='utf-8') as file:
    entita_filtrate = [line.strip() for line in file if line.strip()]

print(entita_filtrate)

['Tecnica', 'Licenza', 'Ordine', 'Ingrediente', 'Piatto', 'Ristorante', 'Pianeta']


In [11]:
# Carica il file CSV
df_domande = pd.read_csv(domande)
testo_domande = df_domande.to_string()  # Converti il CSV in testo
entita_domande = estrai_relazioni(testo=testo_domande, categorie=entita_filtrate, llm=llm)
print("Entità CSV:", entita_domande)

Entità CSV: {'CONTIENE': ['Piatto', 'Ingrediente'], 'PREPARATO': ['Piatto', 'Tecnica'], 'OFFERTA': ['Ristorante', 'Piatto'], 'SITUATO': ['Ristorante', 'Pianeta'], 'RICHIEDE': ['Piatto', 'Licenza'], 'APPARTIENE': ['Ordine', 'Piatto']}


In [13]:
# Carica il file CSV
df_pianeti = pd.read_csv(pianeti)
testo_pianeti = df_pianeti.to_string()  # Converti il CSV in testo
entita_pianeti = estrai_relazioni(testo=testo_pianeti, categorie=entita_filtrate, llm=llm)
print("Entità CSV:", entita_pianeti)

Entità CSV: {'DISTANTE': ['Pianeta', 'Pianeta']}


In [15]:
# Esegui l'analisi del PDF
testo_manuale = estrai_testo_pdf(manuale_cucina)
entita_manuale = estrai_relazioni(testo=testo_manuale, categorie=entita_filtrate, llm=llm)
print("Entità PDF:", entita_manuale)

Entità PDF: {'utilizza': ['Tecnica', 'Ingrediente'], 'prepara': ['Tecnica', 'Piatto'], 'contiene': ['Piatto', 'Ingrediente'], 'situato': ['Ristorante', 'Pianeta'], 'serve': ['Ristorante', 'Piatto'], 'richiede': ['Tecnica', 'Licenza'], 'appartiene_piatto_ord': ['Piatto', 'Ordine'], 'appartiene_ristorante_ord': ['Ristorante', 'Ordine'], 'regola': ['Ordine', 'Licenza'], 'implementa': ['Ordine', 'Tecnica'], 'influenzato': ['Piatto', 'Pianeta']}


In [19]:
# Esegui l'analisi del PDF
testo_codice = estrai_testo_pdf(codice_galattico)
entita_codice = estrai_relazioni(testo=testo_codice, categorie=entita_filtrate, llm=llm)
print("Entità PDF:", entita_codice)

Entità PDF: {'requires': ['Tecnica', 'Licenza'], 'belongs_to': ['Ordine', 'Pianeta'], 'prepares': ['Tecnica', 'Piatto'], 'located_on': ['Ristorante', 'Pianeta'], 'contains': ['Piatto', 'Ingrediente'], 'regulated_by': ['Ingrediente', 'Licenza'], 'implements': ['Ristorante', 'Tecnica'], 'follows': ['Ordine', 'Tecnica']}


In [23]:
# Esegui l'analisi del PDF
testo_menu = estrai_testo_pdf(menu_esempio)
entita_menu = estrai_relazioni(testo=testo_menu, categorie=entita_filtrate, llm=llm)
print("Entità PDF:", entita_menu)

Entità PDF: {'contiene': [['Ristorante', 'Piatto'], ['Piatto', 'Ingrediente']], 'preparato': [['Piatto', 'Tecnica']], 'situato': [['Ristorante', 'Pianeta']], 'possiede': [['Ristorante', 'Licenza']]}


In [73]:
# Esempio di utilizzo:
testo_blog1 = estrai_testo_html(blog_1)
entita_blog1 = estrai_schema(testo=testo_blog1, llm=llm)
print("Entità PDF:", entita_blog1)


Entità PDF: {'Ristorante': 35.0, 'Piatti': 25.0, 'Ingrediente': 20.0, 'Chef': 10.0, 'Tecnica': 10.0}


In [74]:
# Esempio di utilizzo:
testo_blog2 = estrai_testo_html(blog_2)
entita_blog2 = estrai_schema(testo=testo_blog2, llm=llm)
print("Entità PDF:", entita_blog2)

Entità PDF: {'Ristorante': 30.0, 'Ingrediente': 25.0, 'Piatto': 20.0, 'Tecnica': 10.0, 'Chef': 5.0, 'Pianeta': 5.0, 'Critico': 3.0, 'Voto': 2.0}


In [75]:
json_list = [entita_codice, entita_domande, entita_manuale, entita_menu, entita_pianeti, entita_blog1, entita_blog2]
punteggi_combinati = combina_punteggi(json_list, llm)
print("Punteggi combinati:", punteggi_combinati)

Punteggi combinati: {'Sostanza': 30.0, 'Tecnica': 26.25, 'Licenza': 22.75, 'Ordine': 10.0, 'Ingrediente': 28.125, 'Piatto': 40.0, 'Ristorante': 29.0, 'Pianeta': 40.0, 'Cottura': 15.0, 'Preparazione': 8.0, 'Galassia': 5.0, 'Menu': 12.0, 'Chef': 8.333333333333334, 'Critico': 3.0, 'Voto': 2.0}


In [78]:
# Filtra le entità con punteggio superiore a 20
entita_filtrate = [entita for entita, punteggio in punteggi_combinati.items() if punteggio >= 10]

print(entita_filtrate)

['Sostanza', 'Tecnica', 'Licenza', 'Ordine', 'Ingrediente', 'Piatto', 'Ristorante', 'Pianeta', 'Cottura', 'Menu']


In [84]:
entita_filtrate.remove('Sostanza')
entita_filtrate.remove('Menu')
entita_filtrate.remove('Cottura')

ValueError: list.remove(x): x not in list

In [86]:
with open('entita_filtrate.txt', 'w') as file:
    for entita in entita_filtrate:
        file.write(f"{entita}\n")